In [1]:
import sys
import csv
from langdetect import detect
import nltk
from nltk.corpus import stopwords
import string
from collections import defaultdict
import matplotlib.pyplot as plt
import numpy as np
import nltk
import pandas as pd
nltk.download('wordnet')
from nltk.stem.wordnet import WordNetLemmatizer
lem = WordNetLemmatizer()
import os.path
from os import path

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/francisbian/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
#print(string.punctuation)
#stop_words = set(stopwords.words('english'))
#stop_words.add('verse')
#print(stop_words)

## Here I take songs of 1960 as an example

### Let's combine the two lists based on hotlist_100 list's id. Set features to null if not found in spotify list
### I kept the title of songs from both lists. That is because the title from spotify seems to be more accurate, while the title from hotlist seems to be easier for searching

In [12]:
hotlist_csv = pd.read_csv('../newVersionOfLyrics/lyricsHotListTopNew_{}.csv'.format(1960))
spotify_csv = pd.read_csv('../spotify_features_csvs/lyricsHotListTopNew_{}.csv'.format(1960))
combined_csv = pd.merge(hotlist_csv, spotify_csv, how='left', on='Unnamed: 0')

In [13]:
combined_csv

,Unnamed: 0,title,lyrics,artistsFromLyrics,artists,date,rank,cnt,song_name,song_link,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,artist_genres,album_genres
0,0,Last Date,It's over our love affair\nToo late now I find...,Floyd Cramer,Floyd Cramer,1960,"[3, 41, 2, 24, 2, 21, 2, 39, 2, 70, 4, 94, 8, ...",19,Last Date - Instrumental,https://open.spotify.com/track/7rbvjRc6ZTVA3GU...,...,-12.026,1.0,0.0289,0.858000,0.958000,0.1500,0.613,103.818,"['easy listening', 'honky-tonk piano', 'nashvi...",[]
1,1,A Thousand Stars,"1\n\nI celebrate myself, and sing myself,\nAnd...",Walt Whitman,Kathy Young With The Innocents,1960,"[4, 7, 3, 5, 5, 7, 9, 22, 37, 67]",10,A Thousand Stars (Originally Performed by Kath...,https://open.spotify.com/track/5iunW48Iw3Xplzu...,...,-18.272,1.0,0.0400,0.859000,0.155000,0.1080,0.359,98.512,['karaoke'],[]
2,2,Exodus,"This land is mine, God gave this land to me\nT...",Ferrante & Teicher,Ferrante & Teicher,1960,"[5, 6, 10, 30, 40, 51, 71]",7,"Exodus (Theme From Otto Preminger's ""Exodus"")",https://open.spotify.com/track/4pKUJwmq9mYcs1K...,...,-11.842,0.0,0.0259,0.857000,0.497000,0.0638,0.246,98.226,['classic soundtrack'],[]
3,3,North To Alaska,Way up north (north to Alaska)\nWay up north (...,Johnny Horton,Johnny Horton,1960,"[6, 4, 5, 6, 7, 12, 12, 18, 23, 31, 37, 59, 64...",15,North To Alaska*,https://open.spotify.com/track/3JLJ0n5ZFd8oOwv...,...,-14.436,1.0,0.0390,0.718000,0.000000,0.3480,0.916,82.909,"['country', 'cowboy western', 'rock-and-roll']",[]
4,4,Many Tears Ago,They said your cheating heart will make me cry...,Connie Francis,Connie Francis,1960,"[7, 8, 8, 11, 14, 19, 49, 79]",8,Many Tears Ago,https://open.spotify.com/track/4tFENy9kf1zjzRn...,...,-3.466,1.0,0.0294,0.792000,0.000000,0.2610,0.860,127.125,"['adult standards', 'brill building pop', 'eas...",[]
5,5,You're Sixteen,"!\nYou come on like a dream, peaches and cream...",Johnny Burnette,Johnny Burnette,1960,"[8, 9, 12, 17, 21, 32, 41, 62, 94]",9,You're Sixteen,https://open.spotify.com/track/5NOITvzXKvrqsRq...,...,-8.861,1.0,0.0351,0.797000,0.000000,0.0698,0.749,76.269,"['brill building pop', 'rock-and-roll', 'rocka...",[]
6,6,Sailor (Your Home Is The Sea),Ueber Rio und Shanghai\nUeber Bali und Hawaii\...,Lolita,Lolita,1960,"[9, 5, 6, 9, 12, 24, 34, 65, 73, 76]",10,Sailor (Your Home Is The Sea),https://open.spotify.com/track/3l477StNmG8dGEw...,...,-10.041,1.0,0.0325,0.826000,0.000040,0.3500,0.388,102.537,[],[]
7,7,"Corinna, Corinna","I love Corinna, tell the world I do\nI love Co...",Ray Peterson,Ray Peterson,1960,"[10, 27, 42, 71, 84, 96]",6,"Corinna, Corinna",https://open.spotify.com/track/1jnbu1z52a8fmU3...,...,-11.804,1.0,0.0314,0.825000,0.000000,0.1440,0.615,128.923,"['adult standards', 'brill building pop', 'dee...",[]
8,8,He Will Break Your Heart,"[Curtis Mayfield, Calvin Carter and Jerry Butl...",Jerry Butler,Jerry Butler,1960,"[11, 10, 7, 7, 11, 15, 23, 35, 63]",9,He Will Break Your Heart,https://open.spotify.com/track/2ZJZcqM60OwgqTO...,...,-9.095,0.0,0.0425,0.220000,0.000000,0.4460,0.692,130.939,"['beach music', 'brill building pop', 'chicago...",[]
9,9,Lonely Teenager,I wanna go home where I belong\n'Cause now I'm...,Dion and The Belmonts,Dion,1960,"[12, 12, 18, 21, 32, 34, 36, 50, 70, 72, 82]",11,Lonely Teenager,https://open.spotify.com/track/3heOkAfstVuhK57...,...,-11.971,0.0,0.0295,0.613000,0.000362,0.0891,0.700,70.846,"['adult standards', 'brill building pop', 'doo...",[]


### After we do extraction of the lyrics

In [14]:
# For now, I skip the songs whose lyrics is not English or length is smaller than 200 characters

def extract_lyrics(combined_csv):
    combined_csv.insert(3, 'words', np.empty((len(combined_csv), 0)).tolist())
    combined_csv.insert(4, 'lyrics_length', [0]*len(combined_csv))
    for i in range(len(combined_csv)):
        lyric = combined_csv.loc[i, 'lyrics']
        if pd.isnull(combined_csv.loc[i, 'lyrics']):
            continue
        if len(lyric) < 100 or len(lyric) > 10000:
            continue
            
        if detect(lyric) != 'en':
            continue
        li = []
        combined_csv.loc[i, 'valid'] = 1
        for r in lyric.splitlines():
            # Skip sth like '[Instrument]',['Bridge']
            if '[' in r or '(' in r: 
                continue
            for w in r.split():
                combined_csv.loc[i, 'lyrics_length'] += 1
                #if w in stop_words:
                #    continue;
                word = ''.join([ch for ch in w if ch not in string.punctuation]).lower()
                word = lem.lemmatize(word,"v")
                combined_csv.loc[i, 'words'].append(word)


In [15]:
combined_csv.insert(1, 'valid', 0)
extract_lyrics(combined_csv)

In [9]:
combined_csv

,Unnamed: 0,valid,title,words,lyrics_length,lyrics,artistsFromLyrics,artists,date,rank,...,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,artist_genres,album_genres
0,0,0,Last Date,"[its, over, our, love, affair, too, late, now,...",138,It's over our love affair\nToo late now I find...,Floyd Cramer,Floyd Cramer,1960,"[3, 41, 2, 24, 2, 21, 2, 39, 2, 70, 4, 94, 8, ...",...,-12.026,1.0,0.0289,0.858000,0.958000,0.1500,0.613,103.818,"['easy listening', 'honky-tonk piano', 'nashvi...",[]
1,1,0,A Thousand Stars,"[1, i, celebrate, myself, and, sing, myself, a...",15468,"1\n\nI celebrate myself, and sing myself,\nAnd...",Walt Whitman,Kathy Young With The Innocents,1960,"[4, 7, 3, 5, 5, 7, 9, 22, 37, 67]",...,-18.272,1.0,0.0400,0.859000,0.155000,0.1080,0.359,98.512,['karaoke'],[]
2,2,0,Exodus,"[this, land, be, mine, god, give, this, land, ...",125,"This land is mine, God gave this land to me\nT...",Ferrante & Teicher,Ferrante & Teicher,1960,"[5, 6, 10, 30, 40, 51, 71]",...,-11.842,0.0,0.0259,0.857000,0.497000,0.0638,0.246,98.226,['classic soundtrack'],[]
3,3,0,North To Alaska,"[north, to, alaska, , go, north, the, rush, be...",257,Way up north (north to Alaska)\nWay up north (...,Johnny Horton,Johnny Horton,1960,"[6, 4, 5, 6, 7, 12, 12, 18, 23, 31, 37, 59, 64...",...,-14.436,1.0,0.0390,0.718000,0.000000,0.3480,0.916,82.909,"['country', 'cowboy western', 'rock-and-roll']",[]
4,4,0,Many Tears Ago,"[they, say, your, cheat, heart, will, make, me...",178,They said your cheating heart will make me cry...,Connie Francis,Connie Francis,1960,"[7, 8, 8, 11, 14, 19, 49, 79]",...,-3.466,1.0,0.0294,0.792000,0.000000,0.2610,0.860,127.125,"['adult standards', 'brill building pop', 'eas...",[]
5,5,0,You're Sixteen,"[, you, come, on, like, a, dream, peach, and, ...",148,"!\nYou come on like a dream, peaches and cream...",Johnny Burnette,Johnny Burnette,1960,"[8, 9, 12, 17, 21, 32, 41, 62, 94]",...,-8.861,1.0,0.0351,0.797000,0.000000,0.0698,0.749,76.269,"['brill building pop', 'rock-and-roll', 'rocka...",[]
6,6,0,Sailor (Your Home Is The Sea),"[ueber, rio, und, shanghai, ueber, bali, und, ...",42,Ueber Rio und Shanghai\nUeber Bali und Hawaii\...,Lolita,Lolita,1960,"[9, 5, 6, 9, 12, 24, 34, 65, 73, 76]",...,-10.041,1.0,0.0325,0.826000,0.000040,0.3500,0.388,102.537,[],[]
7,7,0,"Corinna, Corinna","[i, love, corinna, tell, the, world, i, do, i,...",133,"I love Corinna, tell the world I do\nI love Co...",Ray Peterson,Ray Peterson,1960,"[10, 27, 42, 71, 84, 96]",...,-11.804,1.0,0.0314,0.825000,0.000000,0.1440,0.615,128.923,"['adult standards', 'brill building pop', 'dee...",[]
8,8,0,He Will Break Your Heart,"[he, dont, love, you, like, i, love, you, if, ...",180,"[Curtis Mayfield, Calvin Carter and Jerry Butl...",Jerry Butler,Jerry Butler,1960,"[11, 10, 7, 7, 11, 15, 23, 35, 63]",...,-9.095,0.0,0.0425,0.220000,0.000000,0.4460,0.692,130.939,"['beach music', 'brill building pop', 'chicago...",[]
9,9,0,Lonely Teenager,"[i, wanna, go, home, where, i, belong, cause, ...",130,I wanna go home where I belong\n'Cause now I'm...,Dion and The Belmonts,Dion,1960,"[12, 12, 18, 21, 32, 34, 36, 50, 70, 72, 82]",...,-11.971,0.0,0.0295,0.613000,0.000362,0.0891,0.700,70.846,"['adult standards', 'brill building pop', 'doo...",[]


In [105]:
combined_csv.to_csv('../combined_dataset/lyrics&features_{}.csv'.format(1960))

## Let's combine all the dataset we have and save to the directory

In [17]:
for year in range(1960, 2021):
    hotlist_csv = pd.read_csv('../newVersionOfLyrics/lyricsHotListTopNew_{}.csv'.format(year))
    combined_csv = hotlist_csv
    # Skip 
    if path.exists('../spotify_features_csvs/lyricsHotListTopNew_{}.csv'.format(year)):
        spotify_csv = pd.read_csv('../spotify_features_csvs/lyricsHotListTopNew_{}.csv'.format(year))
        combined_csv = pd.merge(hotlist_csv, spotify_csv, how='left', on='Unnamed: 0')
    
    extract_lyrics(combined_csv)
    combined_csv.to_csv('../combined_dataset/lyrics&features_{}.csv'.format(year)) 